In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
from torch.utils import data
%matplotlib inline
import torchvision.transforms as standard_transforms
from torch.utils.data import DataLoader
import random
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from PIL import Image, ImageOps, ImageFilter
import numbers
import pdb
import pandas as pd
import torch.nn.functional as F
import torchvision.transforms.functional as F2
import torch.utils.model_zoo as model_zoo
from glob import glob
from torchvision import transforms, models
from torch.utils.data.dataloader import default_collate
from torch import optim
from torch.nn import Module
import time
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

Code can be run both on laptop and on GCP.

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: True 


unzip data

In [25]:
!unzip data.zip

Archive:  data.zip
replace data/bayes/train/IMG_290.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


## Models

We use 2 nn, a CSRNet and a VGG19 extented.

#### VGG19 extended

In [5]:
__all__ = ['vgg19']
model_urls = {
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
}

class VGGExtended(nn.Module):
    def __init__(self, features):
        super(VGGExtended, self).__init__()
        self.features = features
        self.reg_layer = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, 1)
        )

    def forward(self, x):
        x = self.features(x)
        x = F.upsample_bilinear(x, scale_factor=2)
        x = self.reg_layer(x)
        return torch.abs(x)


def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512]
}

def vgg19():
    """VGG 19-layer model (configuration "E")
        model pre-trained on ImageNet
    """
    model = VGGExtended(make_layers(cfg['E']))
    model.load_state_dict(model_zoo.load_url(model_urls['vgg19']), strict=False)
    return model

#### CSRNet

In [6]:
class CSRNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CSRNet, self).__init__()
        self.seen = 0
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512,256,128,64]
        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat,in_channels = 512,dilation = True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        if not load_weights:
            mod = models.vgg16(pretrained = True)
            self._initialize_weights()
            self.frontend.load_state_dict(mod.features[0:23].state_dict())
            
    def forward(self,x):
        size = x.size()
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        x = F.upsample(x, size = size[2:])
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0) 

## Processing helpers

In [7]:
def random_cropBayes(im_h, im_w, crop_h, crop_w):
    res_h = im_h - crop_h
    res_w = im_w - crop_w
    i = random.randint(0, res_h)
    j = random.randint(0, res_w)
    return i, j, crop_h, crop_w


def cal_innner_area(c_left, c_up, c_right, c_down, bbox):
    inner_left = np.maximum(c_left, bbox[:, 0])
    inner_up = np.maximum(c_up, bbox[:, 1])
    inner_right = np.minimum(c_right, bbox[:, 2])
    inner_down = np.minimum(c_down, bbox[:, 3])
    inner_area = np.maximum(inner_right-inner_left, 0.0) * np.maximum(inner_down-inner_up, 0.0)
    return inner_area

## Datasets

#### Ground Truth dataset

In [8]:
class GTDataset(data.Dataset):
    def __init__(self, data_path, mode, main_transform=None, img_transform=None, gt_transform=None):
        self.img_path = data_path + '/img'
        self.gt_path = data_path + '/den'
        self.data_files = [filename for filename in os.listdir(self.img_path) \
                           if os.path.isfile(os.path.join(self.img_path,filename))]
        self.num_samples = len(self.data_files) 
        self.main_transform=main_transform  
        self.img_transform = img_transform
        self.gt_transform = gt_transform     
    
    def __getitem__(self, index):
        fname = self.data_files[index]
        img, den = self.read_image_and_gt(fname)      
        if self.main_transform is not None:
            img, den = self.main_transform(img,den) 
        if self.img_transform is not None:
            img = self.img_transform(img)         
        if self.gt_transform is not None:
            den = self.gt_transform(den)               
        return img, den

    def __len__(self):
        return self.num_samples

    def read_image_and_gt(self,fname):
        img = Image.open(os.path.join(self.img_path,fname))
        if img.mode == 'L':
            img = img.convert('RGB')

        den = pd.read_csv(os.path.join(self.gt_path,os.path.splitext(fname)[0] + '.csv'), sep=',',header=None).values
        
        den = den.astype(np.float32, copy=False)    
        den = Image.fromarray(den)  
        return img, den    

    def get_num_samples(self):
        return self.num_samples

#### Bayes method Dataset

In [9]:
class BayesDataset(data.Dataset):
    def __init__(self, root_path, crop_size,
                 downsample_ratio, is_gray=False,
                 method='train'):

        self.root_path = root_path
        self.im_list = sorted(glob(os.path.join(self.root_path, '*.jpg')))
        if method not in ['train', 'val', 'test']:
            raise Exception("not implement")
        self.method = method

        self.c_size = crop_size
        self.d_ratio = downsample_ratio
        assert self.c_size % self.d_ratio == 0
        self.dc_size = self.c_size // self.d_ratio

        if is_gray:
            self.trans = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])
        else:
            self.trans = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Pour CSRNet à vérifier si besoin de modif
            ])

    def __len__(self):
        return len(self.im_list)

    def __getitem__(self, item):
        img_path = self.im_list[item]
        gd_path = img_path.replace('jpg', 'npy')
        img = Image.open(img_path).convert('RGB')
        if self.method == 'train':
            keypoints = np.load(gd_path)
            return self.train_transform(img, keypoints)
        else:
            keypoints = np.load(gd_path)
            img = self.trans(img)
            name = os.path.basename(img_path).split('.')[0]
            return img, len(keypoints), name

    def train_transform(self, img, keypoints):
        """random crop image patch and find people in it"""
        
        """
        Les keypoints correspondent aux coordonnées des têtes
        MAIS une troisième coordonnée a été calculée lors du preprocessing des données,
        elle correspont à "dis" et semble important pour calculer pas mal de choses
        """
        
        wd, ht = img.size
        st_size = min(wd, ht)
        assert st_size >= self.c_size
        assert len(keypoints) > 0
        i, j, h, w = random_cropBayes(ht, wd, self.c_size, self.c_size)
        img = F2.crop(img, i, j, h, w)
        
        nearest_dis = np.clip(keypoints[:, 2], 4.0, 128.0)
       
        points_left_up = keypoints[:, :2] - nearest_dis[:, None] / 2.0
        points_right_down = keypoints[:, :2] + nearest_dis[:, None] / 2.0
        bbox = np.concatenate((points_left_up, points_right_down), axis=1)
        inner_area = cal_innner_area(j, i, j+w, i+h, bbox)
        origin_area = nearest_dis * nearest_dis
        ratio = np.clip(1.0 * inner_area / origin_area, 0.0, 1.0)
        mask = (ratio >= 0.3)

        target = ratio[mask]
        keypoints = keypoints[mask]
        keypoints = keypoints[:, :2] - [j, i]  # change coodinate
        if len(keypoints) > 0:
            if random.random() > 0.5:
                img = F2.hflip(img)
                keypoints[:, 0] = w - keypoints[:, 0]
        else:
            if random.random() > 0.5:
                img = F2.hflip(img)
        return self.trans(img), torch.from_numpy(keypoints.copy()).float(), \
               torch.from_numpy(target.copy()).float(), st_size

## DataLoader

#### Loading Data GT

In [10]:
#CSRNet
LABEL_FACTOR = 1


def random_crop_GT(img,den,dst_size):
    # dst_size: ht, wd

    _,ts_hd,ts_wd = img.shape

    x1 = random.randint(0, ts_wd - dst_size[1])//LABEL_FACTOR*LABEL_FACTOR
    y1 = random.randint(0, ts_hd - dst_size[0])//LABEL_FACTOR*LABEL_FACTOR
    x2 = x1 + dst_size[1]
    y2 = y1 + dst_size[0]

    label_x1 = x1//LABEL_FACTOR
    label_y1 = y1//LABEL_FACTOR
    label_x2 = x2//LABEL_FACTOR
    label_y2 = y2//LABEL_FACTOR

    return img[:,y1:y2,x1:x2], den[label_y1:label_y2,label_x1:label_x2]



def share_memory(batch):
    out = None
    if False:
        # If we're in a background process, concatenate directly into a
        # shared memory tensor to avoid an extra copy
        numel = sum([x.numel() for x in batch])
        storage = batch[0].storage()._new_shared(numel)
        out = batch[0].new(storage)
    return out

crop_size = 256

def GT_collate(batch):
    # @GJY 
    r"""Puts each data field into a tensor with outer dimension batch size"""

    transposed = list(zip(*batch)) # imgs and dens
    imgs, dens = [transposed[0],transposed[1]]


    error_msg = "batch must contain tensors; found {}"
    if isinstance(imgs[0], torch.Tensor) and isinstance(dens[0], torch.Tensor):
        
        cropped_imgs = []
        cropped_dens = []
        for i_sample in range(len(batch)):
            _img, _den = random_crop_GT(imgs[i_sample],dens[i_sample],[crop_size,crop_size])
            cropped_imgs.append(_img)
            cropped_dens.append(_den)


        cropped_imgs = torch.stack(cropped_imgs, 0, out=share_memory(cropped_imgs))
        cropped_dens = torch.stack(cropped_dens, 0, out=share_memory(cropped_dens))

        return [cropped_imgs,cropped_dens]

    raise TypeError((error_msg.format(type(batch[0]))))


def loading_data_GT(batch_size=5, num_workers=8):
    mean_std = ([0.410824894905, 0.370634973049, 0.359682112932], [0.278580576181, 0.26925137639, 0.27156367898])
    log_para = 100.
    factor = 1
    # DATA_PATH = "/home/simon/Bureau/framework-crowd-counting/ProcessedData/shanghaitech_part_A"
    DATA_PATH = "data/gt"
    
    
    train_main_transform = Compose([
        RandomHorizontallyFlip()
    ])
    img_transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(*mean_std)
    ])
    gt_transform = standard_transforms.Compose([
        GTScaleDown(factor),
        LabelNormalize(log_para)
    ])

    train_set = GTDataset(DATA_PATH+'/train', 'train',main_transform=train_main_transform, img_transform=img_transform, gt_transform=gt_transform)
    train_loader =None
    if batch_size == 1:
        train_loader = DataLoader(train_set, batch_size=1, shuffle=True, drop_last=True)
    elif batch_size > 1:
        train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, collate_fn=GT_collate, shuffle=True, drop_last=True)
    
    val_set = GTDataset(DATA_PATH+'/val', 'val', main_transform=None, img_transform=img_transform, gt_transform=gt_transform)
    val_loader = DataLoader(val_set, batch_size=1, num_workers=num_workers, shuffle=True, drop_last=False)
    
    test_set = GTDataset(DATA_PATH+'/test', 'test', main_transform=None, img_transform=img_transform, gt_transform=gt_transform)
    test_loader = DataLoader(test_set, batch_size=1, num_workers=num_workers, shuffle=True, drop_last=False)
    
    return train_loader, val_loader, test_loader




#### Loading Data Bayes

In [28]:
#bayes
downsample_ratio = 1 # Mettre à 8 pour le réseau du répo (à 1 pour CSRNet puisque on ne modifie pas la dim avec le réseau)
data_dir = "data/bayes"
#data_dir = "/home/simon/Bureau/framework-crowd-counting/processed_data_bcc/SHHA"
#data_dir = "/Users/VictoRambaud/dev/crowd_counting2/ProcessedData/SHHA"
crop_size = 256
is_gray = False

def train_collate(batch):
    transposed_batch = list(zip(*batch))
    images = torch.stack(transposed_batch[0], 0)
    points = transposed_batch[1]  # the number of points is not fixed, keep it as a list of tensor
    targets = transposed_batch[2]
    st_sizes = torch.FloatTensor(transposed_batch[3])
    return images, points, targets, st_sizes


def loading_data_Bayes(batch_size = 5, num_workers = 8):
    datasets_bayes = {x: BayesDataset(os.path.join(data_dir, x),
                              crop_size,
                              downsample_ratio,
                              is_gray, x) for x in ['train', 'val', 'test']}

    dataloaders_bayes = {x: DataLoader(datasets_bayes[x],
                                collate_fn=(train_collate if x == 'train' else default_collate),
                                batch_size=(batch_size if x == 'train' else 1),
                                shuffle=(True if x == 'train' else False),
                                num_workers=num_workers,
                                pin_memory=(True if x == 'train' else False))
                                for x in ['train', 'val', 'test']}
    
    dataloaders_bayes_test = "To do"
    
    return dataloaders_bayes["train"], dataloaders_bayes["val"], dataloaders_bayes["test"]
    

In [29]:
loading_data_Bayes()

(<torch.utils.data.dataloader.DataLoader at 0x7f7c527a3f50>,
 <torch.utils.data.dataloader.DataLoader at 0x7f7c527a32d0>)

## Bayes : computing losses

In [30]:
class Post_Prob(Module):
    def __init__(self, sigma, c_size, stride, background_ratio, use_background, device):
        super(Post_Prob, self).__init__()
        assert c_size % stride == 0

        self.sigma = sigma
        self.bg_ratio = background_ratio
        self.device = device
        # coordinate is same to image space, set to constant since crop size is same
        self.cood = torch.arange(0, c_size, step=stride,
                                 dtype=torch.float32, device=device) + stride / 2
        self.cood.unsqueeze_(0)
        self.softmax = torch.nn.Softmax(dim=0)
        self.use_bg = use_background

    def forward(self, points, st_sizes):
        num_points_per_image = [len(points_per_image) for points_per_image in points]
        all_points = torch.cat(points, dim=0)

        if len(all_points) > 0:
            x = all_points[:, 0].unsqueeze_(1)
            y = all_points[:, 1].unsqueeze_(1)
            x_dis = -2 * torch.matmul(x, self.cood) + x * x + self.cood * self.cood
            y_dis = -2 * torch.matmul(y, self.cood) + y * y + self.cood * self.cood
            y_dis.unsqueeze_(2)
            x_dis.unsqueeze_(1)
            dis = y_dis + x_dis
            dis = dis.view((dis.size(0), -1))

            dis_list = torch.split(dis, num_points_per_image)
            prob_list = []
            for dis, st_size in zip(dis_list, st_sizes):
                if len(dis) > 0:
                    if self.use_bg:
                        min_dis = torch.clamp(torch.min(dis, dim=0, keepdim=True)[0], min=0.0)
                        d = st_size * self.bg_ratio
                        bg_dis = (d - torch.sqrt(min_dis))**2
                        dis = torch.cat([dis, bg_dis], 0)  # concatenate background distance to the last
                    dis = -dis / (2.0 * self.sigma ** 2)
                    prob = self.softmax(dis)
                else:
                    prob = None
                prob_list.append(prob)
        else:
            prob_list = []
            for _ in range(len(points)):
                prob_list.append(None)
        return prob_list
    
    
class Bay_Loss(Module):
    def __init__(self, use_background, device):
        super(Bay_Loss, self).__init__()
        self.device = device
        self.use_bg = use_background

    def forward(self, prob_list, target_list, pre_density):
        loss = 0
        
        """
            - prob list semble être la listes des p(yn|xm) ie la contribution du pixel xm sur la n-ieme tête
            (les lignes de cette matrice sont de taille 4096 = 64*64)
            - pre density est la prédiction de la densité (sortie du réseau) - de taille 64x64 ici
            - target list a pour longueur le nombre de têtes - correspond aux E[cn] "réel" (le calcul reste un mystère)
            - On obtient les E[cn] estimées grâce à un produit terme à terme de prob_list et pre_density
        """
        
        
        for idx, prob in enumerate(prob_list):  # iterative through each sample
            if prob is None:  # image contains no annotation points
                pre_count = torch.sum(pre_density[idx])
                target = torch.zeros((1,), dtype=torch.float32, device=self.device)
            else:
                N = len(prob)
                if self.use_bg:
                    target = torch.zeros((N,), dtype=torch.float32, device=self.device)
                    target[:-1] = target_list[idx]
                else:
                    target = target_list[idx]
                pre_count = torch.sum(pre_density[idx].view((1, -1)) * prob, dim=1)  # flatten into vector
            
            loss += torch.sum(torch.abs(target - pre_count))
        loss = loss / len(prob_list)
        return loss

## Utils

In [31]:
# ===============================img tranforms============================

class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, mask, bbx=None):
        if bbx is None:
            for t in self.transforms:
                img, mask = t(img, mask)
            return img, mask
        for t in self.transforms:
            img, mask, bbx = t(img, mask, bbx)
        return img, mask, bbx

class RandomHorizontallyFlip(object):
    def __call__(self, img, mask, bbx=None):
        if random.random() < 0.5:
            if bbx is None:
                return img.transpose(Image.FLIP_LEFT_RIGHT), mask.transpose(Image.FLIP_LEFT_RIGHT)
            w, h = img.size
            xmin = w - bbx[:,3]
            xmax = w - bbx[:,1]
            bbx[:,1] = xmin
            bbx[:,3] = xmax
            return img.transpose(Image.FLIP_LEFT_RIGHT), mask.transpose(Image.FLIP_LEFT_RIGHT), bbx
        if bbx is None:
            return img, mask
        return img, mask, bbx



# ===============================label tranforms============================

class LabelNormalize(object):
    def __init__(self, para):
        self.para = para

    def __call__(self, tensor):
        # tensor = 1./(tensor+self.para).log()
        tensor = torch.from_numpy(np.array(tensor))
        tensor = tensor*self.para
        return tensor

    
class GTScaleDown(object):
    def __init__(self, factor=8):
        self.factor = factor

    def __call__(self, img):
        w, h = img.size
        if self.factor==1:
            return img
        tmp = np.array(img.resize((w//self.factor, h//self.factor), Image.BICUBIC))*self.factor*self.factor
        img = Image.fromarray(tmp)
        return img

## Trainers

Dossier à créer sur Google Cloud !

In [32]:
save_dir = "best_model_weights"

Tensorboard settings

In [57]:
from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter()

# image = Image.open('../ProcessedData/SHHA/train/IMG_1.jpg')
# trans1 = transforms.ToTensor()
# img = trans1(image).to(device)
# img = img.unsqueeze(0)

# write to tensorboard

# writer.add_graph(model, img)
# writer.close()



In [43]:
# tensorboard --logdir=runs

#### Trainer GT

In [44]:
LOG_PARA = 100. # C'est quoi ce LOG_PARA ??
seed = 1


class Trainer_GT():
    def __init__(self, dataloader, net, loss, optimizer, validation_frequency=1, max_epoch=100):
        self.train_loader, self.val_loader, self.test_loader = dataloader()
        self.net = net
        self.loss = loss 
        self.optimizer = optimizer
        self.best_mae = 1e20
        self.best_mse = 1e20
        self.epoch = 0
        self.validation_frequency = validation_frequency
        self.max_epoch = max_epoch



    def train(self):
        for epoch in range(0, self.max_epoch):
            self.epoch = epoch
            # si on veut un lr sheduler il faut le mettre là
                
            # training    
            self.train_epoch()

            # validation
            if epoch%self.validation_frequency==0:
                self.validate()
                
        print(f'Train finished | best_mse: {self.best_mse} | best_mae: {self.best_mae}')


    def train_epoch(self): # training for all datasets
        self.net.train()
        epoch_loss = 0
        
        for step, data in enumerate(self.train_loader, 0):
            img, gt_map = data
            img = Variable(img).to(device)
            gt_map = Variable(gt_map).to(device)

            self.optimizer.zero_grad()
            # pred_map = self.net(img, gt_map)
            
            pred_density_map = self.net(img)
            loss = self.loss(pred_density_map, gt_map)
            loss.backward()
            self.optimizer.step()
            
            gt_count = [int(gt_map[i].sum().data / LOG_PARA) for i in range(gt_map.size()[0])]
            pre_count = [int(pred_density_map[i].sum().data/LOG_PARA) for i in range(pred_density_map.size()[0])]
            epoch_loss += float(loss)
            
            print(f'epoch: {self.epoch} | step: {step} | count: {gt_count} | prediction: {pre_count} | loss: {loss}')
        
        writer.add_scalar('train loss GT',
            epoch_loss,
            self.epoch)


    def validate(self):
        epoch_start = time.time()
        self.net.eval()
        epoch_res = []

        for vi, data in enumerate(self.val_loader, 0):
            img, gt_map = data

            with torch.no_grad():
                img = Variable(img).to(device)
                assert img.size(0) == 1
                gt_map = Variable(gt_map).to(device)
                pred_density_map = self.net(img)
                
                pred_cnt = int(gt_map[0].sum().data / LOG_PARA)
                gt_count = int(pred_density_map[0].sum().data/LOG_PARA)
                res = gt_count - pred_cnt
                    
                epoch_res.append(res)


        epoch_res = np.array(epoch_res)
        mse = np.sqrt(np.mean(np.square(epoch_res)))
        mae = np.mean(np.abs(epoch_res))
        
        print('Epoch {} Val, MSE: {:.2f} MAE: {:.2f}, Cost {:.1f} sec'
                     .format(self.epoch, mse, mae, time.time()-epoch_start))
        
        writer.add_scalar('val MAE GT',
                    mae,
                    self.epoch)
        writer.add_scalar('val MSE GT',
                        mse,
                        self.epoch)

        if (2.0 * mse + mae) < (2.0 * self.best_mse + self.best_mae):
            self.best_mse = mse
            self.best_mae = mae
            print("save best mse {:.2f} mae {:.2f} model epoch {}".format(self.best_mse,
                                                                            self.best_mae,
                                                                                 self.epoch))
            torch.save(self.net.state_dict(), os.path.join(save_dir, 'best_model_gt.pth'))


In [ ]:
# Launch GT Train !
lr = 1e-6

gt_net = CSRNet().to(device)
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(gt_net.parameters(), lr=lr)
# optimizer = optim.SGD(gt_net.parameters(), lr=lr, momentum=0.95,weight_decay=5e-4)  

gt_trainer = Trainer_GT(loading_data_GT, gt_net, loss, optimizer, max_epoch=250)
gt_trainer.train()

epoch: 0 | step: 0 | count: [31, 21, 30, 2, 1] | prediction: [0, 0, 0, 0, 0] | loss: 0.020717572420835495
epoch: 0 | step: 1 | count: [49, 11, 0, 42, 55] | prediction: [0, 0, 0, 0, 0] | loss: 0.0355154424905777
epoch: 0 | step: 2 | count: [0, 78, 42, 344, 40] | prediction: [0, 0, 0, 0, 0] | loss: 0.15167298913002014
epoch: 0 | step: 3 | count: [405, 8, 4, 0, 104] | prediction: [0, 0, 0, 0, 0] | loss: 0.3549301028251648
epoch: 0 | step: 4 | count: [20, 40, 16, 24, 20] | prediction: [0, 0, 0, 0, 0] | loss: 0.02362821437418461
epoch: 0 | step: 5 | count: [59, 24, 36, 0, 29] | prediction: [0, 0, 0, 0, 0] | loss: 0.029076790437102318
epoch: 0 | step: 6 | count: [213, 286, 581, 93, 41] | prediction: [0, 0, 0, 0, 0] | loss: 0.7049778699874878
epoch: 0 | step: 7 | count: [25, 69, 56, 47, 87] | prediction: [0, 0, 0, 0, 0] | loss: 0.0628519058227539
epoch: 0 | step: 8 | count: [5, 8, 44, 59, 77] | prediction: [0, 0, 0, 0, 0] | loss: 0.039926838129758835
epoch: 0 | step: 9 | count: [52, 28, 14, 7

epoch: 1 | step: 28 | count: [37, 186, 6, 86, 361] | prediction: [4, 4, 4, 4, 2] | loss: 0.32947012782096863
epoch: 1 | step: 29 | count: [11, 22, 83, 0, 40] | prediction: [4, 3, 6, 2, 4] | loss: 0.03238391503691673
epoch: 1 | step: 30 | count: [1, 30, 80, 24, 74] | prediction: [4, 3, 4, 5, 6] | loss: 0.04066537693142891
epoch: 1 | step: 31 | count: [11, 235, 5, 183, 5] | prediction: [3, 4, 4, 4, 4] | loss: 0.11160626262426376
epoch: 1 | step: 32 | count: [132, 22, 45, 120, 51] | prediction: [7, 6, 5, 3, 5] | loss: 0.0733201876282692
epoch: 1 | step: 33 | count: [53, 9, 5, 375, 44] | prediction: [4, 5, 3, 5, 6] | loss: 0.33575117588043213
epoch: 1 | step: 34 | count: [29, 130, 25, 38, 16] | prediction: [5, 6, 5, 5, 4] | loss: 0.04682972654700279
epoch: 1 | step: 35 | count: [14, 30, 0, 86, 0] | prediction: [5, 6, 3, 8, 2] | loss: 0.025759048759937286
epoch: 1 | step: 36 | count: [34, 249, 26, 8, 1] | prediction: [5, 3, 4, 4, 5] | loss: 0.07798546552658081
epoch: 1 | step: 37 | count: [

epoch: 3 | step: 4 | count: [147, 52, 38, 56, 8] | prediction: [41, 65, 55, 46, 48] | loss: 0.06603101640939713
epoch: 3 | step: 5 | count: [61, 66, 70, 16, 64] | prediction: [45, 56, 29, 49, 61] | loss: 0.05406080186367035
epoch: 3 | step: 6 | count: [17, 20, 14, 1, 60] | prediction: [48, 37, 46, 43, 65] | loss: 0.0231565423309803
epoch: 3 | step: 7 | count: [107, 61, 16, 3, 124] | prediction: [65, 64, 36, 26, 68] | loss: 0.07100804895162582
epoch: 3 | step: 8 | count: [1, 24, 36, 124, 134] | prediction: [45, 43, 46, 58, 74] | loss: 0.05842021852731705
epoch: 3 | step: 9 | count: [27, 130, 34, 123, 5] | prediction: [69, 69, 42, 82, 42] | loss: 0.057912588119506836
epoch: 3 | step: 10 | count: [8, 22, 10, 70, 10] | prediction: [51, 49, 51, 45, 24] | loss: 0.024718990549445152
epoch: 3 | step: 11 | count: [70, 28, 30, 21, 36] | prediction: [62, 45, 45, 52, 55] | loss: 0.03782180696725845
epoch: 3 | step: 12 | count: [72, 111, 5, 444, 29] | prediction: [49, 21, 43, 67, 51] | loss: 0.2041

epoch: 4 | step: 29 | count: [232, 54, 50, 144, 130] | prediction: [77, 53, 71, 93, 62] | loss: 0.1417577862739563
epoch: 4 | step: 30 | count: [64, 9, 14, 12, 35] | prediction: [40, 56, 41, 46, 49] | loss: 0.02591846138238907
epoch: 4 | step: 31 | count: [302, 38, 50, 47, 84] | prediction: [68, 42, 51, 76, 81] | loss: 0.12151595950126648
epoch: 4 | step: 32 | count: [6, 66, 0, 0, 20] | prediction: [43, 85, 20, 4, 73] | loss: 0.022065648809075356
epoch: 4 | step: 33 | count: [16, 9, 382, 0, 71] | prediction: [67, 59, 77, 44, 89] | loss: 0.138270303606987
epoch: 4 | step: 34 | count: [89, 3, 131, 146, 29] | prediction: [76, 50, 62, 22, 65] | loss: 0.0756261795759201
epoch: 4 | step: 35 | count: [48, 7, 44, 6, 496] | prediction: [71, 32, 54, 45, 65] | loss: 0.15555952489376068
epoch: 4 | step: 36 | count: [4, 49, 0, 15, 0] | prediction: [40, 69, 19, 73, 41] | loss: 0.018044380471110344
epoch: 4 | step: 37 | count: [47, 63, 2, 231, 123] | prediction: [60, 81, 42, 85, 101] | loss: 0.087516

epoch: 6 | step: 5 | count: [5, 55, 46, 45, 228] | prediction: [36, 65, 51, 49, 75] | loss: 0.09035713225603104
epoch: 6 | step: 6 | count: [4, 78, 11, 4, 3] | prediction: [41, 75, 50, 35, 40] | loss: 0.022089514881372452
epoch: 6 | step: 7 | count: [1, 58, 147, 126, 69] | prediction: [50, 47, 44, 77, 47] | loss: 0.07642290741205215
epoch: 6 | step: 8 | count: [64, 282, 30, 7, 45] | prediction: [67, 73, 67, 22, 47] | loss: 0.08827582746744156
epoch: 6 | step: 9 | count: [0, 58, 0, 81, 8] | prediction: [29, 53, 26, 51, 43] | loss: 0.027950044721364975
epoch: 6 | step: 10 | count: [18, 53, 75, 30, 23] | prediction: [55, 48, 47, 65, 50] | loss: 0.04655570909380913
epoch: 6 | step: 11 | count: [367, 4, 61, 24, 13] | prediction: [42, 42, 65, 43, 57] | loss: 0.21139541268348694
epoch: 6 | step: 12 | count: [106, 17, 9, 82, 16] | prediction: [76, 43, 46, 71, 46] | loss: 0.04249562695622444
epoch: 6 | step: 13 | count: [52, 7, 9, 198, 28] | prediction: [34, 44, 53, 64, 36] | loss: 0.0887459367

...

epoch: 19 | step: 28 | count: [40, 0, 93, 367, 22] | prediction: [48, 48, 53, 92, 53] | loss: 0.12016250193119049
epoch: 19 | step: 29 | count: [266, 130, 148, 91, 83] | prediction: [59, 63, 77, 71, 69] | loss: 0.18513107299804688
epoch: 19 | step: 30 | count: [15, 31, 22, 136, 38] | prediction: [40, 56, 49, 73, 43] | loss: 0.0464010164141655
epoch: 19 | step: 31 | count: [418, 58, 7, 54, 15] | prediction: [97, 55, 49, 52, 47] | loss: 0.1481553465127945
epoch: 19 | step: 32 | count: [36, 5, 10, 32, 264] | prediction: [60, 41, 47, 53, 101] | loss: 0.08613281697034836
epoch: 19 | step: 33 | count: [149, 19, 73, 14, 53] | prediction: [72, 40, 71, 52, 65] | loss: 0.059517525136470795
epoch: 19 | step: 34 | count: [145, 17, 4, 16, 3] | prediction: [70, 57, 46, 56, 46] | loss: 0.03522429242730141
epoch: 19 | step: 35 | count: [50, 12, 2, 123, 0] | prediction: [55, 45, 44, 83, 31] | loss: 0.04652257263660431
epoch: 19 | step: 36 | count: [3, 39, 6, 61, 163] | prediction: [42, 43, 44, 58, 69] 

epoch: 21 | step: 3 | count: [27, 36, 12, 101, 29] | prediction: [70, 58, 54, 87, 54] | loss: 0.04075601324439049
epoch: 21 | step: 4 | count: [22, 29, 0, 300, 0] | prediction: [53, 60, 26, 97, 33] | loss: 0.1558971405029297
epoch: 21 | step: 5 | count: [38, 21, 53, 6, 153] | prediction: [62, 60, 56, 50, 62] | loss: 0.0640801265835762
epoch: 21 | step: 6 | count: [13, 9, 91, 1, 51] | prediction: [50, 49, 67, 43, 58] | loss: 0.030820636078715324
epoch: 21 | step: 7 | count: [65, 79, 170, 69, 0] | prediction: [65, 65, 95, 89, 9] | loss: 0.0682143047451973
epoch: 21 | step: 8 | count: [11, 4, 117, 22, 107] | prediction: [52, 43, 68, 52, 71] | loss: 0.07579867541790009
epoch: 21 | step: 9 | count: [74, 2, 192, 202, 64] | prediction: [73, 40, 88, 97, 61] | loss: 0.11071637272834778
epoch: 21 | step: 10 | count: [170, 50, 1, 134, 6] | prediction: [51, 59, 49, 66, 56] | loss: 0.0713047981262207
epoch: 21 | step: 11 | count: [39, 29, 122, 326, 45] | prediction: [65, 58, 77, 102, 65] | loss: 0.

epoch: 22 | step: 27 | count: [22, 23, 232, 0, 0] | prediction: [55, 66, 100, 33, 62] | loss: 0.059598349034786224
epoch: 22 | step: 28 | count: [6, 51, 28, 29, 193] | prediction: [45, 65, 51, 57, 102] | loss: 0.05697768181562424
epoch: 22 | step: 29 | count: [15, 36, 4, 17, 88] | prediction: [55, 49, 54, 66, 63] | loss: 0.031984105706214905
epoch: 22 | step: 30 | count: [51, 57, 54, 35, 13] | prediction: [60, 64, 72, 50, 55] | loss: 0.038719501346349716
epoch: 22 | step: 31 | count: [39, 157, 68, 0, 50] | prediction: [62, 85, 69, 31, 63] | loss: 0.06991454213857651
epoch: 22 | step: 32 | count: [33, 27, 0, 31, 1] | prediction: [64, 68, 47, 59, 43] | loss: 0.021159693598747253
epoch: 22 | step: 33 | count: [21, 131, 156, 0, 6] | prediction: [67, 76, 71, 52, 41] | loss: 0.06073020026087761
epoch: 22 | step: 34 | count: [65, 39, 208, 8, 39] | prediction: [76, 55, 100, 44, 57] | loss: 0.06829317659139633
epoch: 22 | step: 35 | count: [15, 21, 13, 53, 42] | prediction: [48, 54, 56, 55, 55]

epoch: 24 | step: 2 | count: [0, 28, 88, 42, 41] | prediction: [41, 55, 65, 65, 69] | loss: 0.039778925478458405
epoch: 24 | step: 3 | count: [0, 22, 8, 55, 38] | prediction: [54, 61, 46, 57, 61] | loss: 0.025824060663580894
epoch: 24 | step: 4 | count: [26, 112, 28, 14, 6] | prediction: [58, 79, 52, 47, 45] | loss: 0.037577927112579346
epoch: 24 | step: 5 | count: [71, 118, 169, 19, 61] | prediction: [70, 81, 75, 50, 59] | loss: 0.08291309326887131
epoch: 24 | step: 6 | count: [7, 50, 58, 18, 63] | prediction: [49, 59, 58, 60, 68] | loss: 0.036987900733947754
epoch: 24 | step: 7 | count: [39, 4, 0, 94, 27] | prediction: [64, 54, 37, 74, 83] | loss: 0.03302765265107155
epoch: 24 | step: 8 | count: [40, 32, 15, 329, 15] | prediction: [63, 58, 53, 90, 44] | loss: 0.16688929498195648
epoch: 24 | step: 9 | count: [101, 8, 90, 4, 88] | prediction: [72, 49, 77, 49, 70] | loss: 0.05037681385874748
epoch: 24 | step: 10 | count: [36, 94, 37, 74, 48] | prediction: [59, 65, 59, 76, 59] | loss: 0.

epoch: 25 | step: 26 | count: [7, 105, 129, 10, 12] | prediction: [51, 73, 70, 50, 52] | loss: 0.04980204626917839
epoch: 25 | step: 27 | count: [6, 57, 5, 10, 36] | prediction: [42, 71, 51, 52, 61] | loss: 0.028319640085101128
epoch: 25 | step: 28 | count: [35, 29, 8, 13, 12] | prediction: [62, 53, 50, 44, 60] | loss: 0.021741697564721107
epoch: 25 | step: 29 | count: [39, 65, 41, 7, 20] | prediction: [68, 61, 60, 39, 54] | loss: 0.03363553062081337
epoch: 25 | step: 30 | count: [64, 16, 26, 24, 81] | prediction: [59, 53, 53, 52, 68] | loss: 0.038284782320261
epoch: 25 | step: 31 | count: [98, 18, 31, 90, 0] | prediction: [84, 54, 57, 61, 43] | loss: 0.04852430149912834
epoch: 25 | step: 32 | count: [6, 33, 5, 39, 223] | prediction: [48, 56, 54, 65, 104] | loss: 0.058334968984127045
epoch: 25 | step: 33 | count: [19, 10, 114, 8, 7] | prediction: [50, 46, 66, 54, 49] | loss: 0.04098635911941528
epoch: 25 | step: 34 | count: [8, 57, 124, 66, 12] | prediction: [48, 56, 76, 72, 43] | loss

#### Trainer Bayes

In [39]:
class Trainer_Bayes():
    def __init__(self, dataloader, net, loss, optimizer,  validation_frequency=1, max_epoch=100):
        self.train_loader, self.val_loader, _ = dataloader()
        self.net = net
        self.loss = loss 
        self.optimizer = optimizer
        self.best_mae = 1e20
        self.best_mse = 1e20
        self.epoch = 0
        self.validation_frequency = validation_frequency
        self.max_epoch = max_epoch
        

    def train(self):
        for epoch in range(0, self.max_epoch):
            self.epoch = epoch

            # training    
            self.train_epoch()

            # validation
            if epoch % self.validation_frequency == 0:
                self.validate()
                
        print(f'Train finished | best_mse: {self.best_mse} | best_mae: {self.best_mae}')
                

    def train_epoch(self):
        self.net.train()
        epoch_loss = 0
        
        for step, (inputs, points, targets, st_sizes) in enumerate(self.train_loader):
            inputs = inputs.to(device)
            st_sizes = st_sizes.to(device)
            gd_count = np.array([len(p) for p in points], dtype=np.float32)
            points = [p.to(device) for p in points]
            targets = [t.to(device) for t in targets]
            
            with torch.set_grad_enabled(True):
                outputs = self.net(inputs)
                prob_list = post_prob(points, st_sizes)
                loss = self.loss(prob_list, targets, outputs)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                N = inputs.size(0) # batch size
                pre_count = torch.sum(outputs.view(N, -1), dim=1).detach().cpu().numpy()
                res = pre_count - gd_count
                epoch_loss += float(loss)
                
                print(f'epoch: {self.epoch} | step: {step} | gd_count: {gd_count} | prediction: {pre_count} | loss: {loss}')
       
        writer.add_scalar('train loss Bayes',
            epoch_loss,
            self.epoch)

    def validate(self):
        epoch_start = time.time()
        self.net.eval()  # Set model to evaluate mode
        epoch_res = []

        # Iterate over data.
        for inputs, count, name in self.val_loader:
            inputs = inputs.to(device)
            # inputs are images with different sizes
            assert inputs.size(0) == 1 # 'the batch size should equal to 1 in validation mode'
            with torch.set_grad_enabled(False):
                outputs = self.net(inputs)
                res = count[0].item() - torch.sum(outputs).item()
                epoch_res.append(res)


        epoch_res = np.array(epoch_res)
        mse = np.sqrt(np.mean(np.square(epoch_res)))
        mae = np.mean(np.abs(epoch_res))

        # ...log the running loss
        writer.add_scalar('val MAE Bayes',
                            mae,
                            self.epoch)
        writer.add_scalar('val MSE Bayes',
                        mse,
                        self.epoch)

        print('Epoch {} Val, MSE: {:.2f} MAE: {:.2f}, Cost {:.1f} sec'
                     .format(self.epoch, mse, mae, time.time()-epoch_start))

        if (2.0 * mse + mae) < (2.0 * self.best_mse + self.best_mae):
            self.best_mse = mse
            self.best_mae = mae
            print("save best mse {:.2f} mae {:.2f} model epoch {}".format(self.best_mse,
                                                                            self.best_mae,
                                                                                 self.epoch))
            torch.save(self.net.state_dict(), os.path.join(save_dir, 'best_model_bayes.pth'))

In [40]:
lr = 0.00001
sigma = 0.1
use_background = False
background_ratio = 1

bayes_net = CSRNet().to(device)
optimizer = optim.Adam(bayes_net.parameters(), lr=lr)

post_prob = Post_Prob(sigma,
                           crop_size,
                           downsample_ratio,
                           background_ratio,
                           use_background,
                           device)
loss = Bay_Loss(use_background, device)

bayes_trainer = Trainer_Bayes(loading_data_Bayes, bayes_net, loss, optimizer, max_epoch=50)
bayes_trainer.train()

epoch: 0 | step: 0 | gd_count: [764.  35. 121.  55.  41.] | prediction: [ 1.1734737  -0.5358329  -2.1619778   0.8747963   0.83166647] | loss: 195.96067810058594
epoch: 0 | step: 1 | gd_count: [ 46.  60.  24.  23. 107.] | prediction: [30.892334 25.346998 38.396698 21.787317 26.02096 ] | loss: 34.24599075317383
epoch: 0 | step: 2 | gd_count: [ 72. 121.  12.  15. 194.] | prediction: [33.293617 49.386833 42.65613  42.45251  45.40686 ] | loss: 71.55133056640625
epoch: 0 | step: 3 | gd_count: [75. 27. 22. 34. 33.] | prediction: [54.414    60.753166 59.412373 49.57749  55.834785] | loss: 31.879133224487305
epoch: 0 | step: 4 | gd_count: [ 49.  22.  64. 217.  79.] | prediction: [45.937218 28.814945 40.03018  42.442444 47.0007  ] | loss: 50.07511520385742
epoch: 0 | step: 5 | gd_count: [247.  95.  82.  69.   1.] | prediction: [27.493107 32.38881  43.11589  36.847103 26.54713 ] | loss: 76.5000991821289
epoch: 0 | step: 6 | gd_count: [143.  52.  33. 113. 170.] | prediction: [32.694454 32.587234 3

epoch: 1 | step: 6 | gd_count: [224.  46. 182.  60.  17.] | prediction: [126.595604  58.205795 155.24347   77.96738   23.088873] | loss: 39.5947265625
epoch: 1 | step: 7 | gd_count: [496. 190.  39.  47. 295.] | prediction: [ 74.255264  78.59016   33.871853  56.757874 186.10313 ] | loss: 133.2845001220703
epoch: 1 | step: 8 | gd_count: [142.   7.  47.  34.  54.] | prediction: [125.91382     6.7408504  46.88152    32.856987   66.66794  ] | loss: 21.045455932617188
epoch: 1 | step: 9 | gd_count: [254.   9.  83.   4. 300.] | prediction: [142.62837     5.8829107 102.90018     2.9487004 240.65588  ] | loss: 49.03963851928711
epoch: 1 | step: 10 | gd_count: [166.  61. 245.  69.  16.] | prediction: [69.87828  25.91167  81.17125  20.37563   7.554511] | loss: 70.96468353271484
epoch: 1 | step: 11 | gd_count: [210.  76. 239.  59. 120.] | prediction: [ 51.785645  60.7522   134.7822    32.487625 101.76242 ] | loss: 66.97354888916016
epoch: 1 | step: 12 | gd_count: [ 77.  18.  18. 128.  52.] | predi

epoch: 2 | step: 10 | gd_count: [129.  53.  35.  32.   5.] | prediction: [81.909874 31.894379 45.75521  43.799004 11.534307] | loss: 22.69855308532715
epoch: 2 | step: 11 | gd_count: [  3.  18.  33.  97. 118.] | prediction: [  6.7417803  19.756748   33.16746    88.46847   106.46133  ] | loss: 16.880205154418945
epoch: 2 | step: 12 | gd_count: [15. 63.  3. 19. 86.] | prediction: [ 9.57955   50.385887   2.7935371 18.87825   89.87189  ] | loss: 14.299142837524414
epoch: 2 | step: 13 | gd_count: [ 21. 202.  66.  53.  57.] | prediction: [ 26.40318  148.8189    34.932262  38.110413  27.678074] | loss: 26.815332412719727
epoch: 2 | step: 14 | gd_count: [ 19.   4.  53. 192.   4.] | prediction: [ 18.715279    7.1553926  53.43641   104.874985    1.6479177] | loss: 23.211610794067383
epoch: 2 | step: 15 | gd_count: [131.  49.  38.  25. 283.] | prediction: [ 81.42499   19.083776  21.6892     9.817995 269.53445 ] | loss: 36.66590118408203
epoch: 2 | step: 16 | gd_count: [ 34.  32.  24. 270. 156.] |

epoch: 3 | step: 15 | gd_count: [ 90.  39. 201.  25.  34.] | prediction: [42.46851  22.81834  97.86815   9.874573 22.816462] | loss: 33.63671875
epoch: 3 | step: 16 | gd_count: [172.  11.   0.  21.  62.] | prediction: [47.961506 10.592445 13.482571 14.606262 55.67321 ] | loss: 33.142024993896484
epoch: 3 | step: 17 | gd_count: [58. 32. 38.  1. 80.] | prediction: [55.991917 24.378296 32.564877  7.662466 40.954285] | loss: 15.885504722595215
epoch: 3 | step: 18 | gd_count: [36.  0. 40. 54. 39.] | prediction: [33.962883  5.096125 38.595676 26.98198  39.15174 ] | loss: 13.570784568786621
epoch: 3 | step: 19 | gd_count: [15. 50. 47. 64. 69.] | prediction: [16.835644 57.327473 56.256527 70.040955 75.5211  ] | loss: 20.264509201049805
epoch: 3 | step: 20 | gd_count: [238.  19. 131.  31.  59.] | prediction: [101.20888   15.154784  81.00964   31.806545  51.720516] | loss: 43.66622543334961
epoch: 3 | step: 21 | gd_count: [ 68.  98. 142.  23.  21.] | prediction: [ 41.37961  112.50323   82.65572 

epoch: 4 | step: 20 | gd_count: [ 70.  86.  34. 208.  18.] | prediction: [ 34.14246   71.51673   36.140625 155.66164   16.549746] | loss: 31.723691940307617
epoch: 4 | step: 21 | gd_count: [ 68. 139.  64.  13.  42.] | prediction: [ 54.373363 115.21193   50.45674   19.552317  25.45932 ] | loss: 20.823532104492188
epoch: 4 | step: 22 | gd_count: [ 83.  44. 152.   0. 114.] | prediction: [58.087433  36.921623  86.85936   -1.4357965 78.23598  ] | loss: 29.713117599487305
epoch: 4 | step: 23 | gd_count: [ 26.  20.  16.  17. 134.] | prediction: [ 14.610017   9.311903 153.45158    8.082491 136.5534  ] | loss: 41.78010177612305
epoch: 4 | step: 24 | gd_count: [ 35.   6.  79.  12. 414.] | prediction: [ 24.112793    7.5841756  47.01381     2.7801871 285.01257  ] | loss: 37.77671432495117
epoch: 4 | step: 25 | gd_count: [ 23.  53. 129.   3.  32.] | prediction: [ 14.369108   49.384735  120.3902      3.4530272  25.286526 ] | loss: 11.095305442810059
epoch: 4 | step: 26 | gd_count: [ 16.  21. 109.  2

...

epoch: 44 | step: 44 | gd_count: [ 63. 508.   7.  43.  46.] | prediction: [ 52.29851   371.0645      4.6852627   8.2727785  44.113106 ] | loss: 51.01762008666992
epoch: 44 | step: 45 | gd_count: [ 36.   0.   1.  36. 747.] | prediction: [ 23.079218    2.1540504   2.5178523  40.550972  704.09985  ] | loss: 62.30221176147461
epoch: 44 | step: 46 | gd_count: [ 69.  38.  60. 110.  51.] | prediction: [80.25795  28.088118 69.46735  95.00923  59.452168] | loss: 19.534963607788086
epoch: 44 | step: 47 | gd_count: [183. 109.  18.  36.  23.] | prediction: [185.20291   66.22389   14.729334  17.771889  24.36396 ] | loss: 21.58099937438965
Epoch 44 Val, MSE: 181.67 MAE: 108.52, Cost 8.5 sec
epoch: 45 | step: 0 | gd_count: [ 53.  33.  56.  47. 161.] | prediction: [ 44.670776  46.705864  38.18737   45.802174 155.37244 ] | loss: 20.471593856811523
epoch: 45 | step: 1 | gd_count: [ 79. 175. 293.  96.  12.] | prediction: [ 65.951584 148.44305  280.41833   65.14988    5.465749] | loss: 36.35939025878906
e

Epoch 45 Val, MSE: 264.78 MAE: 133.39, Cost 8.5 sec
epoch: 46 | step: 0 | gd_count: [32.  6. 68. 32. 32.] | prediction: [19.734245   3.9846282 45.74904   23.044815  27.742914 ] | loss: 8.96134090423584
epoch: 46 | step: 1 | gd_count: [ 12.  29. 121. 133.  88.] | prediction: [ 11.575041  25.188353 152.91068   74.001785  79.09802 ] | loss: 32.83039474487305
epoch: 46 | step: 2 | gd_count: [ 34.  20.  27. 110.  26.] | prediction: [ 26.19965   19.022255  20.306757 126.481476  23.743889] | loss: 10.730093955993652
epoch: 46 | step: 3 | gd_count: [15. 34. 45. 75. 33.] | prediction: [10.856439 26.695833 42.865067 93.18663  33.280018] | loss: 14.823206901550293
epoch: 46 | step: 4 | gd_count: [30. 28. 39. 20. 69.] | prediction: [25.781158 22.13929  49.55521  16.06704  74.68388 ] | loss: 9.478469848632812
epoch: 46 | step: 5 | gd_count: [ 24.  56. 171. 160.  70.] | prediction: [ 17.255625  39.454735 135.71597  136.52231   58.2566  ] | loss: 26.316516876220703
epoch: 46 | step: 6 | gd_count: [24

epoch: 47 | step: 4 | gd_count: [  5. 141. 244.  13. 143.] | prediction: [  3.6434398  97.99263   220.79813     9.878296  109.683525 ] | loss: 34.03936767578125
epoch: 47 | step: 5 | gd_count: [ 35.   9. 223.  52.  13.] | prediction: [ 26.25079    4.527002 230.09622   35.825367   8.294422] | loss: 21.445423126220703
epoch: 47 | step: 6 | gd_count: [ 14.  91.  37.   0. 159.] | prediction: [ 10.598456   62.88378    27.731201    2.4118419 106.6203   ] | loss: 21.46632194519043
epoch: 47 | step: 7 | gd_count: [ 78.  28.  37. 107.   3.] | prediction: [57.87015   12.253308  21.37495   60.135487  -3.0687742] | loss: 17.7388916015625
epoch: 47 | step: 8 | gd_count: [37. 14. 72. 39.  5.] | prediction: [23.085995    7.3158092  51.279255   32.536648    0.79461455] | loss: 9.597407341003418
epoch: 47 | step: 9 | gd_count: [ 51. 100.  32.  33.  16.] | prediction: [41.00747  85.63037  23.587662 26.341484  5.492161] | loss: 13.393445014953613
epoch: 47 | step: 10 | gd_count: [29. 31. 45. 55. 29.] | p

epoch: 48 | step: 9 | gd_count: [ 42. 110. 123.  22. 210.] | prediction: [ 38.855804  89.20523  125.151535  18.13282  150.8963  ] | loss: 29.790775299072266
epoch: 48 | step: 10 | gd_count: [300.   7.  30.  22.  16.] | prediction: [218.45581     7.8913636  30.678436   16.480572   10.76992  ] | loss: 24.60469627380371
epoch: 48 | step: 11 | gd_count: [167.  47.  41. 128.  89.] | prediction: [165.1922    39.638123  40.859947 150.37123   78.98662 ] | loss: 26.00154685974121
epoch: 48 | step: 12 | gd_count: [129. 206.  41.  70.  28.] | prediction: [112.6374   208.96814   36.97307   48.55235   29.167767] | loss: 24.51656150817871
epoch: 48 | step: 13 | gd_count: [158.  77.  19.   0.   8.] | prediction: [128.90483    68.443436   17.169554   -2.0871403   3.1561136] | loss: 14.840819358825684
epoch: 48 | step: 14 | gd_count: [173.  26.  35. 146. 109.] | prediction: [116.43529   21.039232  31.507774 110.90381   99.24677 ] | loss: 28.99724769592285
epoch: 48 | step: 15 | gd_count: [273.  34.  27

epoch: 49 | step: 13 | gd_count: [ 99. 180. 120.  92. 191.] | prediction: [ 91.92086  192.33786   97.424995  91.18108  119.72247 ] | loss: 39.369049072265625
epoch: 49 | step: 14 | gd_count: [27.  0. 12.  2. 72.] | prediction: [22.725733   -0.86657465  9.640396    1.6718063  63.424854  ] | loss: 5.855239391326904
epoch: 49 | step: 15 | gd_count: [121.  86.  24.  19.  17.] | prediction: [101.21237    87.19855    24.34212    13.5581875  16.700148 ] | loss: 12.446191787719727
epoch: 49 | step: 16 | gd_count: [ 73. 130. 140.  29.  44.] | prediction: [ 72.60195  116.94959  126.419586  29.55299   36.302616] | loss: 21.887250900268555
epoch: 49 | step: 17 | gd_count: [ 50.   0.  20.  57. 210.] | prediction: [ 50.90311      0.47870982  21.503788    61.325115   169.13383   ] | loss: 22.606725692749023
epoch: 49 | step: 18 | gd_count: [ 27.  27.   9. 184.   1.] | prediction: [ 26.430435   24.800194    9.728435  161.88425     2.3574572] | loss: 14.671191215515137
epoch: 49 | step: 19 | gd_count: 

## Results on test set

#### Test GT

In [ ]:
_, _, test_dataloader = loading_data_GT()

gt_net.load_state_dict(torch.load(os.path.join(save_dir, 'best_model_gt.pth'), device))
gt_net.eval()
errors = []


for vi, data in enumerate(test_dataloader, 0):
    img, gt_map = data

    with torch.no_grad():
        img = Variable(img).to(device)
        assert img.size(0) == 1
        gt_map = Variable(gt_map).to(device)
        pred_density_map = gt_net(img)
        pred_cnt = int(gt_map[0].sum().data / LOG_PARA)
        gt_count = int(pred_density_map[0].sum().data/LOG_PARA)
        error = gt_count - pred_cnt
        print(vi, error, gt_count, pred_cnt)

        errors.append(error)


errors = np.array(errors)
mse = np.sqrt(np.mean(np.square(errors)))
mae = np.mean(np.abs(errors))

log_str = 'Final Test: mae {}, mse {}'.format(mae, mse)
print(log_str)

#### Test Bayes

In [41]:
_, _, test_dataloader = loading_data_Bayes()

bayes_net.load_state_dict(torch.load(os.path.join(save_dir, 'best_model_bayes.pth'), device))
errors = []

for inputs, count, name in test_dataloader:
    inputs = inputs.to(device)
    assert inputs.size(0) == 1
    with torch.set_grad_enabled(False):
        outputs = bayes_net(inputs)
        error = count[0].item() - torch.sum(outputs).item()
        print(name, error, count[0].item(), torch.sum(outputs).item())
        errors.append(error)

errors = np.array(errors)
mse = np.sqrt(np.mean(np.square(errors)))
mae = np.mean(np.abs(errors))
log_str = 'Final Test: mae {}, mse {}'.format(mae, mse)
print(log_str)


('IMG_1',) -202.23348999023438 172 374.2334899902344
('IMG_10',) 88.87063598632812 502 413.1293640136719
('IMG_100',) -24.1907958984375 389 413.1907958984375
('IMG_101',) -192.61929321289062 211 403.6192932128906
('IMG_102',) -32.866729736328125 223 255.86672973632812
('IMG_103',) -151.6229248046875 431 582.6229248046875
('IMG_104',) -328.8505859375 1175 1503.8505859375
('IMG_105',) 34.80546569824219 265 230.1945343017578
('IMG_106',) 158.375244140625 1232 1073.624755859375
('IMG_107',) 106.19613647460938 289 182.80386352539062
('IMG_108',) 1.051666259765625 182 180.94833374023438
('IMG_109',) 7.990203857421875 379 371.0097961425781
('IMG_11',) 63.0145263671875 1068 1004.9854736328125
('IMG_110',) 266.922607421875 1021 754.077392578125
('IMG_111',) -93.29608154296875 452 545.2960815429688
('IMG_112',) -96.02096557617188 256 352.0209655761719
('IMG_113',) -166.2392578125 66 232.2392578125
('IMG_114',) -18.207733154296875 141 159.20773315429688
('IMG_115',) -81.3922119140625 1191 1272.39

('IMG_77',) 19.209259033203125 207 187.79074096679688
('IMG_78',) 9.372802734375 271 261.627197265625
('IMG_79',) 41.407928466796875 186 144.59207153320312
('IMG_8',) 633.2510986328125 1326 692.7489013671875
('IMG_80',) 11.48370361328125 157 145.51629638671875
('IMG_81',) -77.814453125 356 433.814453125
('IMG_82',) 28.390884399414062 217 188.60911560058594
('IMG_83',) -13.132156372070312 218 231.1321563720703
('IMG_84',) -47.686431884765625 122 169.68643188476562
('IMG_85',) 5.472686767578125 97 91.52731323242188
('IMG_86',) 92.7808837890625 579 486.2191162109375
('IMG_87',) 31.757720947265625 417 385.2422790527344
('IMG_88',) 0.8637466430664062 86 85.1362533569336
('IMG_89',) -26.011383056640625 383 409.0113830566406
('IMG_9',) 111.75747680664062 371 259.2425231933594
('IMG_90',) 545.5450439453125 2256 1710.4549560546875
('IMG_91',) -7.98394775390625 101 108.98394775390625
('IMG_92',) 29.25 1366 1336.75
('IMG_93',) 28.058013916015625 255 226.94198608398438
('IMG_94',) 11.5452117919921

In [ ]:
## Fonction Victor - à garder pour le github final

def test_bayes(net, test_data, has_loader=False):
    """
    net : the trained network
    has_loader : if false, we are just giving a single input and want to get its results, 
        else we are giving a dataloader
    test_data : just input (np.array) and count if has_loader == False, data_loader if not
    """
    
    net.eval()  # Set model to evaluate mode
    
    if not has_loader:
        img, count = test_data[0], len(test_data[1])
        # img must be a np array
        img = img.to(device)
        img = np.asarray(img)
        #the chanels must be in first position in order to work
        if img.shape[0] != 3:
            img = np.moveaxis(img, (0,1,2), (1,2,0))
        img = torch.Tensor(img).unsqueeze(0)
        with torch.set_grad_enabled(False):
            outputs = net(img)
            res = np.abs(count - torch.sum(outputs).item())
        return outputs, res
    
    else:
        full_res = []

        # Iterate over data.
        for inputs, count, name in test_data:
            print(name)
            inputs = inputs.to(device)
            # inputs are images with different sizes
            assert inputs.size(0) == 1, 'the batch size should equal to 1 in test mode'
            with torch.set_grad_enabled(False):
                outputs = net(inputs)
                res = count[0].item() - torch.sum(outputs).item()
                full_res.append(res)


        res = np.array(full_resres)
        mse = np.sqrt(np.mean(np.square(res)))
        mae = np.mean(np.abs(res))

        print('MSE: {:.2f} MAE: {:.2f}'
                     .format(mse, mae))
        return